In [2]:
%pip install dash==2.14.2

  Using cached dash-2.14.2-py3-none-any.whl.metadata (11 kB)
Using cached dash-2.14.2-py3-none-any.whl (10.2 MB)
  Attempting uninstall: dash
    Found existing installation: dash 4.0.0
    Uninstalling dash-4.0.0:
      Successfully uninstalled dash-4.0.0
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dash-extensions 2.0.4 requires dash>=3.0.0, but you have dash 2.14.2 which is incompatible.
dash-leaflet 1.1.3 requires dash>=3.0.0, but you have dash 2.14.2 which is incompatible.


In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table, callback_context
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os
import sys 

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from CRUD import animalShelter

###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "SNHU1234"
HOST = 'localhost'
PORT = 27017
DB = 'aac'
COL = 'animals'

shelter = animalShelter(username, password, HOST, PORT, DB, COL)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Adds the grazioso Salvare Logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    # html.Div(id='hidden-div', style={'display':'none'}),
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
            style={'width':'200px', 'height':'200px', 'display': 'block', 'margin': '0 auto'}),
    html.H6('Levi Bajuscik - Client-Side Authentification'),  # Unique Identifier
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Hr(),
    # Create buttons for rescue type
    html.Div(className='buttonRow',
            style={'display' : 'flex'},
                children=[html.Button(id='submit-button-one', n_clicks=0, children='Water Rescue'),
                         html.Button(id='submit-button-two', n_clicks=0, children='Moutain Rescue'),
                         html.Button(id='submit-button-three', n_clicks=0, children='Disaster Rescue'),
                         html.Button(id='submit-button-four', n_clicks=0, children='Reset')
                         ]),
    # show current selected button message
    html.Div(id='status-message', style={'fontSize': '20px', 'color': 'Black'}),

    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
                         editable = False,
                         filter_action="native",
                         sort_action="native",
                         sort_mode="multi",
                         column_selectable=False,
                         row_selectable="single",
                         row_deletable=False,
                         selected_columns=[],
                         selected_rows=[0],
                         page_action="native",
                         page_current=0,
                         page_size=10, 
                         style_table={'overflowX': 'auto'},  # Sidescroll to view all columns
    ),
                     
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



# define inputs for callback   
@app.callback(
    [Output('datatable-id', 'data'),
     Output('status-message','children')],
    [Input('submit-button-one', 'n_clicks'),
    Input('submit-button-two', 'n_clicks'),
    Input('submit-button-three', 'n_clicks'),
    Input('submit-button-four', 'n_clicks')]
    )
    
def update_dashboard(button1, button2, button3, button4):
    
    ctx = callback_context
    
    if not ctx.triggered: # if no button is clicked, outputs all documents
        query = {}
        msg = "Showing all results:"
    else:
        button_id = ctx.triggered[0]['prop_id'].split('.')[0]
        if button_id == 'submit-button-one': 
            query = {
                "breed": {"$regex": "Labrador Retriever Mix|Chesa Bay Retr|Newfoundland"},
                "sex_upon_outcome": "Intact Female",
                "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
            } 
            msg = "Showing results for Water Rescue:"
        elif button_id == 'submit-button-two':
            query = {
                "breed": {"$regex": "German Shepherd|Alaskan Malamute|Old English Sheepdog|Siberian Husky|Rottweiler"},
                "sex_upon_outcome": "Intact Male",
                "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
            }
            msg = "Showing results for Mountain Rescue:"
        elif button_id == 'submit-button-three':
            query = {
                "breed": {"$regex": "Doberman Pinsch|German Shepherd|Golden Retriever|Bloodhound|Rottweiler"},
                "sex_upon_outcome": "Intact Male",
                "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
            }
            msg = "Showing results for Disaster Rescue:"
        elif button_id == 'submit_button-four':
            query = {}
            msg = "Showing all results:"
        else:
            query = {}
            msg = ""
            
    # reads the query data     
    df = pd.DataFrame.from_records(shelter.read(query))
    
    # Remove the MongoDB '_id' column if it exists so that the datatable won’t crash
    if '_id' in df.columns:
        df.drop(columns=['_id'], inplace=True)

    # Return the updated records to the Dash DataTable and message
    return df.to_dict('records'), msg


# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):

    if not viewData:
        return [html.Div("No data available for pie chart")]
        
    # Pie chart for data
    df = pd.DataFrame.from_records(viewData)

    if 'breed' not in df.columns or df['breed'].isna().all():
        return [html.Div("No breed data available.")]

    fig = px.pie(df, names='breed', title='Preferred Dog Breeds')
    return [dcc.Graph(figure=fig)]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
    if not viewData:
        return [html.Div("No data for map")]
    
    dff = pd.DataFrame.from_dict(viewData)
    
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    lat = float(dff.at[row, 'location_lat'])
    lon = float(dff.at[row, 'location_long'])
    tooltip_text = str(dff.at[row, 'breed']) if 'breed' in dff.columns else ""
    popup_text = str(dff.at[row, 'name']) if 'name' in dff.columns else ""

    #map_key = f"map-{raw_lat:.6f}-{raw_lon:.6f}"

    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '500px', 'height': '500px'}, center=[lat, lon], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[lat, lon], children=[
                dl.Tooltip(tooltip_text),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(popup_text)
                ])
            ])
        ])
    ]

app.run_server(debug=True)

C:\Users\Levi\AppData\Local\Programs\Python\Python311\Lib\site-packages\dash\dash.py:538: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



Dash app running on http://127.0.0.1:8050/
